### 동물 분류
https://drive.google.com/file/d/1_d8RcCM21XneorFe_m4939erMkCnccS1/view?usp=drive_linkk

In [1]:
# with문으로 파일을 열고 f로 할당
with open('./datasets/animals/translate.py') as f:
    # 파일을 내용을 읽어와 content에 저장
    content = f.readline()
    # 중괄호 사이의 문장을 찾아서 가져오고
    # eval을 사용하여 문자열 안에 갇혀있는 문장을 실제 딕셔너리로 변환
    contents1 = eval(content[content.index('{'):content.index('}') + 1])
    # 키와 값을 반대로 변경하여 저장
    contents2 = {v : k for k, v in contents1.items()}

print(contents1, contents2, sep='\n')

{'cane': 'dog', 'cavallo': 'horse', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'pecora': 'sheep', 'scoiattolo': 'squirrel', 'dog': 'cane', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'spider': 'ragno', 'squirrel': 'scoiattolo'}
{'dog': 'cane', 'horse': 'cavallo', 'elephant': 'elefante', 'butterfly': 'farfalla', 'chicken': 'gallina', 'cat': 'gatto', 'cow': 'mucca', 'sheep': 'pecora', 'squirrel': 'scoiattolo', 'cane': 'dog', 'elefante': 'elephant', 'farfalla': 'butterfly', 'gallina': 'chicken', 'gatto': 'cat', 'mucca': 'cow', 'ragno': 'spider', 'scoiattolo': 'squirrel'}


In [2]:
from glob import glob
import os

# 이미지가 저장되어 있는 경로 지정
root = './datasets/animals/original/'

# 해당 경로를 통해 이미지 폴더를 찾아옴
# glob: 파일 경로와 이름 패턴을 사용하여 파일을 찾을 수 있게 해주는 모듈
# *: 모든 문자열을 대체
directories = glob(os.path.join(root, '*'))

# 원래 정방향인데 에러나면 역방향으로 찾음
for directory in directories:
    try:
        # 디렉터리 이름을 해당하는 번역된 이름으로 변경
        os.rename(directory, os.path.join(root, contents1[directory[directory.rindex('\\') + 1:]]))
    except KeyError as e:
        # KeyError가 발생하면 번역된 이름이 아닌 원래 이름으로 변경
        os.rename(directory, os.path.join(root, contents2[directory[directory.rindex('\\') + 1:]]))

In [3]:
# 이미지가 저장되어 있는 경로 지정
root = './datasets/animals/original/'

# 해당 경로를 통해 이미지 폴더를 찾아옴
directories = glob(os.path.join(root, '*'))
# 폴더 이름 저장할 초기 list 생성
directory_names = []

for directory in directories:
    # 디렉토리의 이름을 찾아와서 list에 저장
    # rindex: 문자열에서 특정 문자 또는 부분 문자열의 마지막으로 발생하는 인덱스를 반환하는 메서드
    directory_names.append(directory[directory.rindex('\\') + 1:])

print(directory_names)

['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']


In [4]:
# 이미지가 저장되어 있는 경로 지정
root = './datasets/animals/original/'

# 이미지 별 폴더 안 이미지들의 파일명 변경
for name in directory_names:
    # os.listdir(): 지정된 디렉토리 내의 파일 목록 가져오기
    # enumerate(): 함수를 사용하여 파일 목록에서 파일 이름과 해당 파일의 인덱스를 반환
    for i, file_name in enumerate(os.listdir(os.path.join(root, name))):
        # 이전 파일의 전체 경로
        old_file = os.path.join(root + name + '/', file_name)
        # 신규 파일 전체 경로 작성
        new_file = os.path.join(root + name + '/', name + str(i + 1) + '.png')

        # 이전 파일의 이름을 신규 파일로 변경
        os.rename(old_file, new_file)

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지의 픽셀 값을 0에서 255 사이에서 0에서 1 사이의 값으로 조정
# ImageDataGenerator: 이미지 데이터를 증강하고 전처리하는 데 사용되는 클래스
image_data_generator = ImageDataGenerator(rescale=1./255)

# 디렉토리에서 이미지를 가져와 배치로 변환
# target_size: 픽셀 크기 지정(150*150)
# batch_size: 매개변수는 각 배치에 포함될 이미지 수를 지정
# class_mode: 분류 작업을 수행할 때 이미지 레이블을 생성하는 방법을 지정
generator = image_data_generator.flow_from_directory(root, target_size=(150, 150), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 26179 images belonging to 10 classes.
{'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [6]:
import pandas as pd

# 파일 경로와 타겟값을 가지고 새로운 데이터 프레임 생성
a_df = pd.DataFrame({'file_paths': generator.filepaths, 'targets': generator.classes})
a_df

,file_paths,targets
0,./datasets/animals/original/butterfly\butterfl...,0
1,./datasets/animals/original/butterfly\butterfl...,0
2,./datasets/animals/original/butterfly\butterfl...,0
3,./datasets/animals/original/butterfly\butterfl...,0
4,./datasets/animals/original/butterfly\butterfl...,0
...,...,...
26174,./datasets/animals/original/squirrel\squirrel9...,9
26175,./datasets/animals/original/squirrel\squirrel9...,9
26176,./datasets/animals/original/squirrel\squirrel9...,9
26177,./datasets/animals/original/squirrel\squirrel9...,9


In [7]:
# 경로 중 \\(역슬래시)로 되어 있는 부분을 /(슬래시)로 변경
a_df.loc[:, 'file_paths'] = a_df.file_paths.apply(lambda x: x.replace('\\', '/'))
a_df

,file_paths,targets
0,./datasets/animals/original/butterfly/butterfl...,0
1,./datasets/animals/original/butterfly/butterfl...,0
2,./datasets/animals/original/butterfly/butterfl...,0
3,./datasets/animals/original/butterfly/butterfl...,0
4,./datasets/animals/original/butterfly/butterfl...,0
...,...,...
26174,./datasets/animals/original/squirrel/squirrel9...,9
26175,./datasets/animals/original/squirrel/squirrel9...,9
26176,./datasets/animals/original/squirrel/squirrel9...,9
26177,./datasets/animals/original/squirrel/squirrel9...,9


In [8]:
from sklearn.model_selection import train_test_split

# 데이터 세트 분리
# 문제/정답 및 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = \
train_test_split(a_df.file_paths, a_df.targets, stratify=a_df.targets, test_size=0.2, random_state=124)

# 정답(타겟) 비중 확인
print(y_train.value_counts())
print(y_test.value_counts())

targets
4    3890
8    3857
2    2478
6    2098
0    1690
3    1493
9    1490
7    1456
1    1334
5    1157
Name: count, dtype: int64
targets
4    973
8    964
2    620
6    525
0    422
3    373
9    372
7    364
1    334
5    289
Name: count, dtype: int64


In [9]:
from sklearn.model_selection import train_test_split

# 검증용 데이터 세트 분리
X_train, X_val, y_train, y_val = \
train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=124)

# 정답(타겟) 비중 확인
print(y_train.value_counts())
print(y_val.value_counts())

targets
4    3112
8    3086
2    1982
6    1678
0    1352
3    1194
9    1192
7    1165
1    1067
5     926
Name: count, dtype: int64
targets
4    778
8    771
2    496
6    420
0    338
3    299
9    298
7    291
1    267
5    231
Name: count, dtype: int64


In [10]:
import shutil

# 경로 지정
root = './datasets/animals/'

for file_path in X_train:
    # 이미지 파일의 원본 디렉터리 경로를 animal_dir에 저장
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    # 이미지 파일을 복사할 훈련용 디렉터리의 경로를 destination에 저장
    destination = os.path.join(root, 'train/' + animal_dir)

    # 훈련용 디렉터리가 없는 경우 해당 디렉터리를 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

     # 이미지 파일을 훈련용 디렉터리로 복사
    shutil.copy2(file_path, destination)

In [11]:
import shutil

# 경로 지정
root = './datasets/animals/'

for file_path in X_val:
    # 이미지 파일의 원본 디렉터리 경로 저장
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    # 이미지 파일을 복사할 훈련용 디렉터리의 경로 저장
    destination = os.path.join(root, 'validation/' + animal_dir)

    # 훈련용 디렉터리가 없는 경우 해당 디렉터리를 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

    # 이미지 파일을 훈련용 디렉터리로 복사
    shutil.copy2(file_path, destination)

In [12]:
import shutil

# 경로 지정
root = './datasets/animals/'

for file_path in X_test:
    # 이미지 파일의 원본 디렉터리 경로 저장
    animal_dir = file_path[len(root + 'original/'): file_path.rindex('/')]
    # 이미지 파일을 복사할 훈련용 디렉터리의 경로 저장
    destination = os.path.join(root, 'test/' + animal_dir)

    # 훈련용 디렉터리가 없는 경우 해당 디렉터리를 생성
    if not os.path.exists(destination):
        os.makedirs(destination)

    # 이미지 파일을 훈련용 디렉터리로 복사
    shutil.copy2(file_path, destination)